In [4]:
import tensorflow as tf
import pandas as pd
import keras
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [9]:
names = ["class", "msg"]
# Cargar los datos desde archivos TSV
train_data = pd.read_csv('train-data.tsv', sep='\t', names= names)
valid_data = pd.read_csv('valid-data.tsv', sep='\t',names= names)

# Convertir etiquetas 'ham' y 'spam' a 0 y 1
train_data['class'] = train_data['class'].map({'ham': 0, 'spam': 1})
valid_data['class'] = valid_data['class'].map({'ham': 0, 'spam': 1})

# Asegúrate de que los archivos tienen dos columnas: 'review' y 'sentiment'
x_train = train_data['msg'].values
y_train = train_data['class'].values.astype(int)  # Convertir a enteros
x_valid = valid_data['msg'].values
y_valid = valid_data['class'].values.astype(int)  # Convertir a enteros

# One-hot encoding de y_train
#y_train_onehot = to_categorical(y_train)

# One-hot encoding de y_test
#y_test_onehot = to_categorical(y_test)


In [10]:
# Parámetros
VOCAB_SIZE = 10000
MAXLEN = 250

# Tokenizar el texto
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(x_train)

# Convertir texto a secuencias de enteros
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_valid_seq = tokenizer.texts_to_sequences(x_valid)

# Padding de las secuencias
x_train_pad = pad_sequences(x_train_seq, maxlen=MAXLEN)
x_valid_pad = pad_sequences(x_valid_seq, maxlen=MAXLEN)


In [34]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 32))
model.add(LSTM(24, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [35]:
# Entrenar el modelo
model.fit(x_train_pad, y_train, epochs=10, batch_size=32, validation_data=(x_valid_pad, y_valid))

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.8616 - loss: 0.4108 - val_accuracy: 0.9749 - val_loss: 0.1218
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9849 - loss: 0.0771 - val_accuracy: 0.9806 - val_loss: 0.0742
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.9936 - loss: 0.0324 - val_accuracy: 0.9885 - val_loss: 0.0491
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9947 - loss: 0.0223 - val_accuracy: 0.9878 - val_loss: 0.0471
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - accuracy: 0.9987 - loss: 0.0085 - val_accuracy: 0.9885 - val_loss: 0.0432
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.9998 - loss: 0.0041 - val_accuracy: 0.9885 - val_loss: 0.0457
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.9999 - loss: 0.0027 - val_accuracy: 0.9892 - val_loss: 0.0422
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.9993 - loss: 0.0040 - val_acc

In [36]:
# Evaluar el modelo
loss, accuracy = model.evaluate(x_valid_pad, y_valid)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')


44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9875 - loss: 0.0560
Loss: 0.05180951952934265
Accuracy: 0.9892241358757019


In [39]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def encode_text(text, tokenizer, maxlen=250):
    tokens = tokenizer.texts_to_sequences([text])
    return pad_sequences(tokens, maxlen=maxlen)[0]

def predict_message(pred_text):
    # Codificar el texto
    encoded_text = encode_text(pred_text, tokenizer)
    
    # Redimensionar la entrada para que sea compatible con el modelo
    pred_input = np.zeros((1, MAXLEN))
    pred_input[0] = encoded_text
    
    # Hacer la predicción
    prediction = model.predict(pred_input)[0][0]
    
    # Determinar la etiqueta basada en el umbral de 0.5
    label = 'spam' if prediction >= 0.2 else 'ham'
    
    return [prediction, label]

# Texto de ejemplo para predicción
pred_text = "how are you doing today?"

# Obtener la predicción
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[0.00032034444, 'ham']


In [41]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
You passed the challenge. Great job!
